# Exploring the distributed information via SPARQL and helper methods
Next, we answer a few typical quations a user would ask when working with the datasets. A user has to know the addresses of the SPARQL endpoints of interest.  
We basically have 2 options:
1. Define the SPARQL-query, then loop over all SPARQL-Endpoints and digest the response
2. Collect the graphs from the Endpoints, put the together into one graph-object and then operate on this object.
   1. collect via SPARQL, store, put together
   2. directly get the turtlefile via the ontodocker API, store locally, then load from the files and put together into one object. 

Set up things to use the mesh and load participant information:

In [1]:
import os
os.environ["REQUESTS_CA_BUNDLE"] = "/etc/ssl/certs/ca-certificates.crt"

import json
import sys
sys.path.append("..")
import helpers

with open('../secrets/participant_registry.json') as f:
    partners = json.load(f, object_hook=lambda d: helpers.RecursiveNamespace(**d))

## SPARQL queries
To make the following notebook easierly readable, we define some queries in `queries.py`. At the moment, they are hard-coded and will only work for these specific versions of pmdCO and the TTO.

In [2]:
from queries import QueryCollection
queries = QueryCollection()

You can find queries which prompt for various information:
- `queries.query_materialDesignation()` For which material is data available?  
This reads out the `pmdco:value` stored under `pmdco:materialDesignation`
- `queries.query_processType()`: What process type was performed on S355?
- `queries.query_orientation()`: In what orientation relative to the rolling direction was each specimen cut?
- `queries.query_deviceAndStandard()`: Which device and standard was used for the measurements?
- `queries.query_specimen()`: What is the specimen ID
- `queries.query_csvurl()`: Under which URL can we find the csv-file?

The queries are defined as class methods. They return a dataclass instance, allowing to easily add new methods(queries) and enabling a dot notation and tab completion. They return the query (as a string) and column headers (as a list of strings), corresponding to the returned values of the resp. query.  
Because a `dataclass` is uesd, it is very easy to make returned contents attributes directly accessible via dot-notation while reducing boilerplate code (defining `__init__` etc.):

In [3]:
query = queries.material_designation()
print("query ="+query.query)
print("")
print("columns = "+str(query.columns))

query =
        PREFIX pmd: <https://w3id.org/pmd/co/>
        SELECT DISTINCT ?p ?matDesVal
        WHERE {
            ?s a pmd:TestPiece .
            ?p pmd:input ?s .
            ?p pmd:characteristic ?matDes .
            ?matDes a pmd:materialDesignation .
            ?matDes pmd:value ?matDesVal .
        }
        ORDER BY ?p
        

columns = ['URI', 'materialDesignation']


## Explore the datasets
Here, we demonstrate how a simple data exploration can be done by sending queries directly to the intances' SPARQL endpoints and looping over instances. We perform queries using the `SPARQLWrapper` module, wrpeed in some convenience methods (from `../helpers.py`)

### Get a first idea
First, we have a look at the accessible endpoints on the instances referenced in `partners`, to know which endpoints are accessible. We do this via http requests to the ontodocker API (sending `GET` to `<address>/api/v1/endpoints`): 

In [4]:
import requests
for key in partners.__dict__:
    address = getattr(partners, key).ontodocker.address
    token = getattr(partners, key).ontodocker.token
    
    headers = {"Authorization": f"Bearer {token}"}
    result = helpers.rectify_endpoints(requests.get(f'{address}/api/v1/endpoints', headers=headers).content.decode())
    print(result)

['http://ontodocker.iwm.pmd.internal/api/v1/jena/pmdco2_tto_example_parallel/sparql', 'http://ontodocker.iwm.pmd.internal/api/v1/jena/test/sparql']
['https://ontodocker.iwt.pmd.internal/api/v1/jena/pmdco2_tto_example_perpendicular/sparql']
['http://ontodocker.mpi-susmat.pmd.internal/api/v1/jena/pmdco2_tto_example_diagonal/sparql']


This gives us an idea of the dataset names. In `heleprs.py`, ther is a method `federated_query()`, which helps to distribute the query among endpoints. To avoid alway querying all endpoints, eventhough they do not contain the dataset of interest, you can pass a list of strings, of which you expect to be in the dataset names (e.g. `"_tto_example_"` in this case). You can also just list the exact names of the datasets to query precisely the correct endpoints. if `datasets = None`, all endpoints are queried.

In [5]:
datasets = None

In [6]:
query = queries.material_designation()
matDes_results = helpers.federated_query(partners, datasets, query.query, query.columns, print_to_screen=True)

Sending query to "http://ontodocker.iwm.pmd.internal/api/v1/jena/pmdco2_tto_example_parallel/sparql". Result:
                                                 URI materialDesignation
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-...                S355
1  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-...                S355
2  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-...                S355
3  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-...                S355

Sending query to "http://ontodocker.iwm.pmd.internal/api/v1/jena/test/sparql". Result:
Empty DataFrame
Columns: [URI, materialDesignation]
Index: []

Sending query to "https://ontodocker.iwt.pmd.internal/api/v1/jena/pmdco2_tto_example_perpendicular/sparql". Result:
                                                 URI materialDesignation
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-...                S355
1  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-...                S355
2  https://w3id.org/pmd/ao/tte/pmdao-tt

You can suppress the output to screen by stting `print_to_screen=False`.  
**FILTERING:** Note the empty oputput for datasets which do not fit the query. This can be avoided by specifying `datasets` more precisely:

In [7]:
datasets = ["pmdco2_tto_example_parallel", "pmdco2_tto_example_perpendicular","pmdco2_tto_example_diagonal"]
matDes_results = helpers.federated_query(partners, datasets, query.query, query.columns, print_to_screen=True)

Sending query to "http://ontodocker.iwm.pmd.internal/api/v1/jena/pmdco2_tto_example_parallel/sparql". Result:
                                                 URI materialDesignation
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-...                S355
1  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-...                S355
2  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-...                S355
3  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-...                S355

Sending query to "https://ontodocker.iwt.pmd.internal/api/v1/jena/pmdco2_tto_example_perpendicular/sparql". Result:
                                                 URI materialDesignation
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-...                S355
1  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-...                S355
2  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-...                S355
3  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-...                S355

Sending query to "http://ontodocker.mpi-su

Again, we can again use the `RecusiveNamespace` class to make this browsable via dot-notation and tab-completion:

In [8]:
matDes_results_rns = helpers.RecursiveNamespace(**matDes_results)

In [9]:
# matDes_results_rns. # uncomment and test tab-completion

The Object returned by 'RecursiveNamespace' is a

In [10]:
print("endpoint =\n"+str(matDes_results_rns.mpi_susmat.ontodocker_mpisusmat.pmdco2_tto_example_diagonal.endpoint))
print("")
print("query ="+str(matDes_results_rns.mpi_susmat.ontodocker_mpisusmat.pmdco2_tto_example_diagonal.query))

endpoint =
http://ontodocker.mpi-susmat.pmd.internal/api/v1/jena/pmdco2_tto_example_diagonal/sparql

query =
        PREFIX pmd: <https://w3id.org/pmd/co/>
        SELECT DISTINCT ?p ?matDesVal
        WHERE {
            ?s a pmd:TestPiece .
            ?p pmd:input ?s .
            ?p pmd:characteristic ?matDes .
            ?matDes a pmd:materialDesignation .
            ?matDes pmd:value ?matDesVal .
        }
        ORDER BY ?p
        


And of course the result of the query is also accessible (as a `pandas` dataframe):

In [11]:
matDes_results_rns.mpi_susmat.ontodocker_mpisusmat.pmdco2_tto_example_diagonal.result

,URI,materialDesignation
0,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-...,S355
1,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-...,S355


Adjust `pandas` to improve readability:

In [12]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.colheader_justify', 'left')
pd.set_option('display.width', 200)

matDes_results_rns.mpi_susmat.ontodocker_mpisusmat.pmdco2_tto_example_diagonal.result

,URI,materialDesignation
0,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-10_process,S355
1,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-9_process,S355


## The actual exploration:
### 1.) What material was tested?

In [13]:
query = queries.material_designation()
material_designation_results = helpers.federated_query(partners, datasets, query.query, query.columns, print_to_screen=True)
material_designation_results_rns = helpers.RecursiveNamespace(**material_designation_results)

Sending query to "http://ontodocker.iwm.pmd.internal/api/v1/jena/pmdco2_tto_example_parallel/sparql". Result:
  URI                                                      materialDesignation
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process  S355              
1  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-2_process  S355              
2  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-3_process  S355              
3  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-4_process  S355              

Sending query to "https://ontodocker.iwt.pmd.internal/api/v1/jena/pmdco2_tto_example_perpendicular/sparql". Result:
  URI                                                      materialDesignation
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-5_process  S355              
1  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-6_process  S355              
2  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-7_process  S355              
3  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-8_process  S3

### 2.) What processes were performed on the specimen? 

In [14]:
query = queries.process_type()
process_type_results = helpers.federated_query(partners, datasets, query.query, query.columns, print_to_screen=True)
process_type_results_rns = helpers.RecursiveNamespace(**process_type_results)

Sending query to "http://ontodocker.iwm.pmd.internal/api/v1/jena/pmdco2_tto_example_parallel/sparql". Result:
  URI                                                      Process type                        
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process  https://w3id.org/pmd/co/TensileTest
1  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-2_process  https://w3id.org/pmd/co/TensileTest
2  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-3_process  https://w3id.org/pmd/co/TensileTest
3  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-4_process  https://w3id.org/pmd/co/TensileTest

Sending query to "https://ontodocker.iwt.pmd.internal/api/v1/jena/pmdco2_tto_example_perpendicular/sparql". Result:
  URI                                                      Process type                        
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-5_process  https://w3id.org/pmd/co/TensileTest
1  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-6_process  https://w3id.org/pmd/co/TensileTest
2  ht

### 3.) In what orientation relative to the rolling direction was each specimen cut?

In [15]:
query = queries.orientation()
orientation_results = helpers.federated_query(partners, datasets, query.query, query.columns, print_to_screen=True)
orientation_results_rns = helpers.RecursiveNamespace(**orientation_results)

Sending query to "http://ontodocker.iwm.pmd.internal/api/v1/jena/pmdco2_tto_example_parallel/sparql". Result:
  URI                                                      cut orientation      
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process  in rolling direction
1  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-2_process  in rolling direction
2  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-3_process  in rolling direction
3  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-4_process  in rolling direction

Sending query to "https://ontodocker.iwt.pmd.internal/api/v1/jena/pmdco2_tto_example_perpendicular/sparql". Result:
  URI                                                      cut orientation                    
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-5_process  perpendicular to rolling direction
1  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-6_process  perpendicular to rolling direction
2  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-7_process  perpendicular to rollin

### 4.) Which device was used for the measurements?

In [16]:
query = queries.extensiometer()
extensiometer_results = helpers.federated_query(partners, datasets, query.query, query.columns, print_to_screen=True)
extensiometer_results_rns = helpers.RecursiveNamespace(**extensiometer_results)

Sending query to "http://ontodocker.iwm.pmd.internal/api/v1/jena/pmdco2_tto_example_parallel/sparql". Result:
  URI                                                      Extensiometer model
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process  HBM               
1  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-2_process  HBM               
2  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-3_process  HBM               
3  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-4_process  HBM               

Sending query to "https://ontodocker.iwt.pmd.internal/api/v1/jena/pmdco2_tto_example_perpendicular/sparql". Result:
  URI                                                      Extensiometer model
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-5_process  HBM               
1  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-6_process  HBM               
2  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-7_process  HBM               
3  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-8_process  HB

### 5.) Which standard was applied?

In [17]:
query = queries.standard()
standard_results = helpers.federated_query(partners, datasets, query.query, query.columns, print_to_screen=True)
standard_results_rns = helpers.RecursiveNamespace(**standard_results)

Sending query to "http://ontodocker.iwm.pmd.internal/api/v1/jena/pmdco2_tto_example_parallel/sparql". Result:
  URI                                                      Extensiometer model Standard         
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process  HBM                 DIN EN ISO 17025
1  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-2_process  HBM                 DIN EN ISO 17025
2  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-3_process  HBM                 DIN EN ISO 17025
3  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-4_process  HBM                 DIN EN ISO 17025

Sending query to "https://ontodocker.iwt.pmd.internal/api/v1/jena/pmdco2_tto_example_perpendicular/sparql". Result:
  URI                                                      Extensiometer model Standard         
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-5_process  HBM                 DIN EN ISO 17025
1  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-6_process  HBM                 DIN EN ISO 170

### 6.) Which ID was assigned to the specimen?

In [18]:
query = queries.specimen_id()
specimen_id_results = helpers.federated_query(partners, datasets, query.query, query.columns, print_to_screen=True)
specimen_id_results_rns = helpers.RecursiveNamespace(**specimen_id_results)

Sending query to "http://ontodocker.iwm.pmd.internal/api/v1/jena/pmdco2_tto_example_parallel/sparql". Result:
  URI                                                      Specimen ID                                                 
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process  https://w3id.org/pmd/ao/tte/testpiece_name/Zx1testpiece/Zx1
1  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-2_process  https://w3id.org/pmd/ao/tte/testpiece_name/Zx2testpiece/Zx2
2  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-3_process  https://w3id.org/pmd/ao/tte/testpiece_name/Zx3testpiece/Zx3
3  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-4_process  https://w3id.org/pmd/ao/tte/testpiece_name/Zx4testpiece/Zx4

Sending query to "https://ontodocker.iwt.pmd.internal/api/v1/jena/pmdco2_tto_example_perpendicular/sparql". Result:
  URI                                                      Specimen ID                                                 
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-5_pr

### 7.) For each process, show the URL to the data file.

In [19]:
query = queries.csv_url()
csv_url_results = helpers.federated_query(partners, datasets, query.query, query.columns, print_to_screen=True)
csv_url_results_rns = helpers.RecursiveNamespace(**csv_url_results)

Sending query to "http://ontodocker.iwm.pmd.internal/api/v1/jena/pmdco2_tto_example_parallel/sparql". Result:
  URI                                                      URL                                                                                                                     
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process  https://github.com/materialdigital/application-ontologies/tree/main/tensile_test_ontology_TTO/data/primary_data/Zx1.csv
1  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-2_process  https://github.com/materialdigital/application-ontologies/tree/main/tensile_test_ontology_TTO/data/primary_data/Zx2.csv
2  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-3_process  https://github.com/materialdigital/application-ontologies/tree/main/tensile_test_ontology_TTO/data/primary_data/Zx3.csv
3  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-4_process  https://github.com/materialdigital/application-ontologies/tree/main/tensile_test_ontology_TTO/data/primary_dat

## Other data: `PrimaryData`, `SecondaryData` and `MetaData`

### 8.) `PrimaryData`
Data acquired before / after a process either refering to the geometry (e.g. width) of the specimen or as a register of the setup (e.g. original gauge length).

In [20]:
query = queries.primary_data()
primary_data_results = helpers.federated_query(partners, datasets, query.query, query.columns, print_to_screen=True)
primary_data_results_rns = helpers.RecursiveNamespace(**primary_data_results)

Sending query to "http://ontodocker.iwm.pmd.internal/api/v1/jena/pmdco2_tto_example_parallel/sparql". Result:
   URI                                                      Quantity                                                                          value    unit                               
0   https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process               https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_crossSectionArea_S0  120.636  http://qudt.org/vocab/unit/MilliM2
1   https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process               https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_crossSectionArea_Su   52.659  http://qudt.org/vocab/unit/MilliM2
2   https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process               https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_crossSectionArea_s1  120.675  http://qudt.org/vocab/unit/MilliM2
3   https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process               https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_crossSectionArea_s2 

### 9.) `SecondaryData`
Characteristic values (e.g. test results) determined by equations or algorithms using primary data and metadata for a process.

In [21]:
query = queries.secondary_data()
secondary_data_results = helpers.federated_query(partners, datasets, query.query, query.columns, print_to_screen=True)
secondary_data_results_rns = helpers.RecursiveNamespace(**secondary_data_results)

Sending query to "http://ontodocker.iwm.pmd.internal/api/v1/jena/pmdco2_tto_example_parallel/sparql". Result:
   URI                                                      Quantity                                                                           value   unit                               
0   https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process                       https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_maximumForce  62.007    http://qudt.org/vocab/unit/kiloN
1   https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process                https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_modulusOfElasticity   194.0   http://qudt.org/vocab/unit/GigaPa
2   https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_percentageElongationAfterFracture     NaN  http://qudt.org/vocab/unit/PERCENT
3   https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process          https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_percentageReductionOfArea

### 10.) `MetaData`
Attributes and additional data concerning the factory, laboratory, the process system and the objects which allow the evaluation of the quality / reliability of the measurements and a systematic search task of a database.

In [22]:
query = queries.metadata()
metadata_results = helpers.federated_query(partners, datasets, query.query, query.columns, print_to_screen=True)
metadata_results_rns = helpers.RecursiveNamespace(**metadata_results)

Sending query to "http://ontodocker.iwm.pmd.internal/api/v1/jena/pmdco2_tto_example_parallel/sparql". Result:
   URI                                                      Quantity                                                                  value unit                               
0   https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_environmentalTemperature  20.0    http://qudt.org/vocab/unit/DEG_C
1   https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process   https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_extensometerGaugeLength  50.0   http://qudt.org/vocab/unit/MilliM
2   https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process           https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_transitionPoint   1.6  http://qudt.org/vocab/unit/PERCENT
3   https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-2_process  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-2_environmentalTemperature  20.0    http://qudt.org/vocab/unit/DEG_C
4   https:

### Further exploration

Queries (at them moment: primary, meta, and secondary data) kan be limitet to only be perfomred for a specified `URI`:

In [23]:
testURI = "https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process"
query = queries.metadata(testURI)
results = helpers.federated_query(partners, datasets, query.query, query.columns, print_to_screen=True)
results_rns = helpers.RecursiveNamespace(**matDes_results)

Sending query to "http://ontodocker.iwm.pmd.internal/api/v1/jena/pmdco2_tto_example_parallel/sparql". Result:
  URI                                                      Quantity                                                                  value unit                               
0  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_environmentalTemperature  20.0    http://qudt.org/vocab/unit/DEG_C
1  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process   https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_extensometerGaugeLength  50.0   http://qudt.org/vocab/unit/MilliM
2  https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process           https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_transitionPoint   1.6  http://qudt.org/vocab/unit/PERCENT

Sending query to "https://ontodocker.iwt.pmd.internal/api/v1/jena/pmdco2_tto_example_perpendicular/sparql". Result:
Empty DataFrame
Columns: [URI, Quantity, value, unit]
Index: []

Sending 

In [24]:
metadata_results_rns.iwm.ontodocker_iwm.pmdco2_tto_example_parallel.result # is a pandas dataframe

,URI,Quantity,value,unit
0,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_environmentalTemperature,20.0,http://qudt.org/vocab/unit/DEG_C
1,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_extensometerGaugeLength,50.0,http://qudt.org/vocab/unit/MilliM
2,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_transitionPoint,1.6,http://qudt.org/vocab/unit/PERCENT
3,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-2_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-2_environmentalTemperature,20.0,http://qudt.org/vocab/unit/DEG_C
4,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-2_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-2_extensometerGaugeLength,50.0,http://qudt.org/vocab/unit/MilliM
5,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-2_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-2_transitionPoint,3.8,http://qudt.org/vocab/unit/PERCENT
6,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-3_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-3_environmentalTemperature,20.0,http://qudt.org/vocab/unit/DEG_C
7,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-3_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-3_extensometerGaugeLength,50.0,http://qudt.org/vocab/unit/MilliM
8,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-3_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-3_transitionPoint,3.8,http://qudt.org/vocab/unit/PERCENT
9,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-4_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-4_environmentalTemperature,20.0,http://qudt.org/vocab/unit/DEG_C


In [25]:
results_test_URI = primary_data_results_rns.iwm.ontodocker_iwm.pmdco2_tto_example_parallel.result.query('URI == @testURI')

In [26]:
results_test_URI

,URI,Quantity,value,unit
0,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_crossSectionArea_S0,120.636,http://qudt.org/vocab/unit/MilliM2
1,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_crossSectionArea_Su,52.659,http://qudt.org/vocab/unit/MilliM2
2,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_crossSectionArea_s1,120.675,http://qudt.org/vocab/unit/MilliM2
3,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_crossSectionArea_s2,120.608,http://qudt.org/vocab/unit/MilliM2
4,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_crossSectionArea_s3,120.625,http://qudt.org/vocab/unit/MilliM2
5,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_finalGaugeLengthAfterFracture_Lu,NaN,http://qudt.org/vocab/unit/MilliM
6,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_originalGaugeLength_L0,60.0,http://qudt.org/vocab/unit/MilliM
7,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_thicknessAfterFracture_au,3.52,http://qudt.org/vocab/unit/MilliM
8,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_thickness_a1,5.993,http://qudt.org/vocab/unit/MilliM
9,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_thickness_a2,5.987,http://qudt.org/vocab/unit/MilliM


In [27]:
results_test_URI.query("Quantity == 'https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_crossSectionArea_S0'")

,URI,Quantity,value,unit
0,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_process,https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_crossSectionArea_S0,120.636,http://qudt.org/vocab/unit/MilliM2


In [28]:
s0_value_ds = results_test_URI.query("Quantity == 'https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_crossSectionArea_S0'")['value']
s0_value_ds

0    120.636
Name: value, dtype: object

In [29]:
s0_unit_ds = results_test_URI.query("Quantity == 'https://w3id.org/pmd/ao/tte/pmdao-tto-tt-S355-1_crossSectionArea_S0'")['unit']
s0_unit_ds

0    http://qudt.org/vocab/unit/MilliM2
Name: unit, dtype: object

In [30]:
s0_value_ds[0] # is a string

'120.636'

In [31]:
import ast
ast.literal_eval(s0_value_ds[0]) # convert to the correct type using abstract sysntax tree

120.636

In [32]:
type(ast.literal_eval(s0_value_ds[0]))

float

In [33]:
type(s0_unit_ds[0])

str